# Assign Weighted Tasks - 1 Person
## Problem
- Given: N tasks
    + Each task has: `start time`, `end time`, `profit`
    + 1 person to do the tasks
    + Task must not be overlapped
- Find the maximum profit subset of tasks such that no 2 tasks overlapped

```C++
struct Task {
    int start_;
    int end_;
    int profit_;
};
int N;
vector<Task> tasks;
```

#### Example 1

```
<!--  Input -->
4
3 10 20
1 2 50
6 19 100
2 100 200

<!--  Output -->
250
```

#### Example 2

```
<!--  Input -->
8
5786 8113 5087
8765 19303 9710
3455 5829 2447
2637 10599 8208
3913 11083 7236
4652 14046 1486
1778 6621 3585
807 10139 8713

<!--  Output -->
14797
```

#### Example 3

```
<!--  Input -->
9
2369 12084 9463
2279 4578 2638
5614 8305 6354
1232 6635 4654
4116 8603 881
2122 3730 9766
8105 18245 4820
10316 12317 7917
4370 11606 1389

<!--  Output -->
24037
```


## Solution
#### Solution DP - O(N*time)
- Ideas: binary knapsack
- **Notes**: tasks[i] start from 1

```C++
int ans;
unordered_map<int, int> dp[1000003];
void get(int i, int time) {
    if(i == N) {
        ans = max(ans, dp[i][time]);
        return;
    }
    if(dp[i].count(time) == NULL) return;

    // Choose i+1 if start time >= previous end time
    if(tasks[i+1].start_ >= time) {
        dp[i+1][tasks[i+1].end_] = max(dp[i+1][tasks[i+1].end_], dp[i][time] + tasks[i+1].profit_);
        get(i+1, tasks[i+1].end_);
    }

    // Not choose i+1
    dp[i+1][time] = max(dp[i+1][time], dp[i][time]);
    get(i+1, time);
}


int sol() {
    // Sort tasks by end time increasing
    sort(tasks.begin()+1, tasks.begin()+N,
        [](const Task &a, const Task &b) {
            return a.end_ < b.end_;
        });

    // DP
    ans = 0;
    dp[0][0] = 0;
    get(0,0);
    return ans;
}
```

#### Solution DP loop - O(N^2)
- Ideas: Greedy choose 1 pre_task to relax
- **Notes**: tasks[i] start from 0

```C++
int dp[1000003];
int sol() {
    // Sort tasks by end time increasing
    sort(tasks.begin() + 1, tasks.end(),
        [](const Task &a, const Task &b) {
            return a.end_ < b.end_;
        });

    // DP
    memset(dp, 0, sizeof(dp));
    dp[0] = 0;

    for(int i=1; i<=N; ++i) {
        // Greedy choose 1 prev_task
        int prev_task = -1;
        for(int j=i-1; j>=1; --j) {
            if(tasks[i].start_ >= tasks[j].end_) {
                prev_task = j;
                break;
            }
        }

        // Relax
        if(prev_task != -1) {
            dp[i] = max(dp[i], tasks[i].profit_ + dp[prev_task]);
        }
        dp[i] = max(dp[i], tasks[i].profit_);
        dp[i] = max(dp[i], dp[i-1]);
    }

    return dp[N];
}
```

#### Solution DP loop with Bin search optimization O(N*logN)
- Ideas: Since end_ is sorted --> use bin search to choose prev_task 
- **Notes**: tasks[i] start from 1

```C++
class Solution {
public:
    struct Task {
        int start_;
        int end_;
        int profit_;
    };
    int jobScheduling(vector<int>& startTime, vector<int>& endTime, vector<int>& profit) {
        int N = startTime.size();

        // Preprocess: 0:start, 1:end, 2:profit
        vector<Task> tasks(1, {0,0,0});
        for(int i=0; i<N; ++i) {
            tasks.push_back( {startTime[i], endTime[i], profit[i]} );
        }

        // Sort by end_time increasing
        sort(tasks.begin()+1, tasks.end(),
            [](const Task &a, const Task &b) {
                return a.end_ < b.end_;
        });

        // knapsack dp
        // dp[i] = max_profit, optimal from 0 -> item i
        vector<int> dp(N+1, 0);
        for(int i=1; i<=N; ++i) {
            // Greedy choose 1 prev_task - Bin search
            int prev_task = -1;
            int l = 1;
            int r = i-1;
            while(l<=r) {
                int m = l + (r-l)/2;
                if(tasks[i].start_ >= tasks[m].end_) {
                    if(tasks[i].start_ >= tasks[m+1].end_) {
                        l = m + 1;
                    } else {
                        prev_task = m;
                        break;
                    }
                } else {
                    r = m - 1;
                }
            }


            // Choose i
            if(prev_task != -1) {
                dp[i] = max(dp[i], tasks[i].profit_ + dp[prev_task]);
            }

            // Not choose i
            dp[i] = max(dp[i], tasks[i].profit_);
            dp[i] = max(dp[i], dp[i-1]);
        }
        return dp[N];
    }
};
```

# [Assign Task - 1 person - max profit](https://leetcode.com/problems/maximum-profit-in-job-scheduling/)
#### Problem
- Have a list of tasks
    + Each task has [start_time, end_time, profit]
- Have 1 worker: Select a set of jobs that return **maximum profit** as possible

#### Example
```
Input: startTime = [1,2,3,3], endTime = [3,4,5,6], profit = [50,10,40,70]
Output: 120
```
<img src="./img/4.png" width="350"/>

```
Input: startTime = [1,2,3,4,6], endTime = [3,5,10,6,9], profit = [20,20,100,70,60]
Output: 150
```
<img src="./img/5.png" width="550"/>


```
Input: startTime = [1,1,1], endTime = [2,3,4], profit = [5,6,4]
Output: 6
```
<img src="./img/6.png" width="400"/>

#### Solution

```Cpp
class Solution {
public:
    struct Task {
        int start_;
        int end_;
        int profit_;
    };
    int jobScheduling(vector<int>& startTime, vector<int>& endTime, vector<int>& profit) {
        int N = startTime.size();

        // Preprocess: 0:start, 1:end, 2:profit
        vector<Task> tasks;
        for(int i=0; i<N; ++i) {
            tasks.push_back( {startTime[i], endTime[i], profit[i]} );
        }

        // Sort by end_time increasing
        sort(tasks.begin(), tasks.end(),
            [](const Task &a, const Task &b) {
                return a.end_ < b.end_;
        });

        // dp[end_time] = max_profit
        map<int, int> dp;
        dp.insert( {0,0} );

        // DP - knapsack
        int ans = 0;
        for(Task &task: tasks) {
            int cur_start_time = task.start_;
            int cur_end_time = task.end_;
            int cur_profit = task.profit_;

            // Use bin search to search for the previous job that (pre_end_time <= cur_start_time)
            int pre_end_time = prev(dp.upper_bound(cur_start_time))->first;

            // Choose i: Greedy move only update if take more profit then the current max
            if(dp[pre_end_time] + cur_profit > ans) {
                dp[cur_end_time] = dp[pre_end_time] + cur_profit;
                ans = max(ans, dp[cur_end_time]);
            }

            // Not choose i: Not needed
            // dp[cur_end_time] = max(dp[cur_end_time], dp[pre_end_time]);
        }
        return ans;
    }
};
```